In [1]:
import os
import math
import numpy as np
# Please put your input file in the same folder as this Jupyter notebook file.
# Please do not change any other code
# You only need to complete the functions with TODO
# You can simply run the Q1/Q2/Q3_test() to test your function

In [2]:
def Q1_input(file_path):
    input_list = []
    with open(file_path, 'r') as file:
        for line in file:
            values = line.strip().split(',')
            if len(values) == 3:
                try:
                    Y = int(values[0])
                    Co = int(values[1])
                    Cg = int(values[2])
                    input_list.append((Y, Co, Cg))
                except ValueError:
                    print(f"Invalid line: {line}")
    return input_list


def Q1_test():
    input_file = "q1_input.txt"
    yco_cg_list = Q1_input(input_file)

    for Y, Co, Cg in yco_cg_list:
        YUV = Q1(Y, Co, Cg)
        print(f"YCoCg: ({Y}, {Co}, {Cg}) -> YUV: (Y={YUV[0]}, U={YUV[1]}, V={YUV[2]})")


In [3]:
def Q1(Y, Co, Cg):
    # TODO:
    # Conversion formula from Y Co Cg to YUV
    YCoCg_RGB_matrix = np.array([
        [1,  1,  0],
        [-1,  1, -1],
        [-1, 1,  1]
    ])

    RGB_YUV_matrix = np.array([
        [0.299, 0.587, 0.114],
        [-0.299, -0.587, 0.886],
        [0.701, -0.587, -0.114]
    ])

    CgYCo = np.array([Cg, Y, Co])
    G, B, R = np.dot(YCoCg_RGB_matrix, CgYCo)

    RGB = np.array([R, G, B])
    Y, U, V = np.dot(RGB_YUV_matrix, RGB)

    return (Y, U, V)


In [5]:
Q1_test()

YCoCg: (100, 5, 5) -> YUV: (Y=101.795, U=-11.794999999999996, V=-1.7950000000000024)


In [6]:
dither_matrix = np.array([
    [0, 3],
    [2, 1]
])

def Q2_input(file_path):
    image = []
    with open(file_path, 'r') as file:
        for line in file:
            row = []
            values = line.strip().split(',')
            for item in values:
                row.append(int(item))
            image.append(row)
    return np.array(image)

def Q2_test():
    file_path = "q2_input.txt"
    input_image = Q2_input(file_path)
    print("input image:\n",input_image)
    dithering_res = dithering(input_image,dither_matrix)
    ordered_dithering_res = ordered_dithering(input_image,dither_matrix)
    print("dithering:\n",dithering_res)
    print("ordered_dithering:\n",ordered_dithering_res)

In [11]:
def dithering(input_image,dither_matrix):
    #TODO:
    # Function for dithering
    ranges = np.linspace(0, 256, 6)
    mapped = np.digitize(input_image, ranges) - 1

    h, w = input_image.shape
    dithered_img = np.zeros((h * 2, w * 2), dtype=np.uint8)

    for y in range(h):
        for x in range(w):
            pixel = mapped[y, x]
            new_block = (pixel > dither_matrix).astype(np.uint8)

            # Store the 2x2 block in the final image
            dithered_img[y*2:y*2+2, x*2:x*2+2] = new_block

    return dithered_img

def ordered_dithering(input_image,dither_matrix):
    #TODO:
    # Function for ordered dithering
    n = np.size(dither_matrix, axis=0)
    ranges = np.linspace(0, 256, 6)
    mapped = np.digitize(input_image, ranges) - 1

    h, w = input_image.shape
    dithered_img = np.zeros((h, w), dtype=np.uint8)
    for y in range(h):
        for x in range(w):
            i = x % n
            j = y % n
            pixel = mapped[y, x]
            dither_entry = dither_matrix[j, i]
            if (pixel > dither_entry):
                dithered_img[y, x] = 1
            else:
                dithered_img[y, x] = 0

    return dithered_img

In [12]:
Q2_test()

input image:
 [[ 12 112 233 100]
 [158 120  10 210]
 [ 31 190  71  81]
 [  1   1   1   1]]
dithering:
 [[0 0 1 0 1 1 1 0]
 [0 0 0 1 1 1 0 0]
 [1 0 1 0 0 0 1 1]
 [1 1 0 1 0 0 1 1]
 [0 0 1 0 1 0 1 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
ordered_dithering:
 [[0 0 1 0]
 [1 1 0 1]
 [0 0 1 0]
 [0 0 0 0]]


In [13]:
def Q3_input(file_path):
    string = []
    with open(file_path, 'r') as file:
        for line in file:
            string.append(line)
    return string


def Q3_test():

    file_path = "q3_input.txt"

    input_list = Q3_input(file_path)
    for input_string in input_list:
        print("input string: ",input_string)
        print("first order entropy:",first_order_entropy(input_string))
        print("second order entropy:",second_order_entropy(input_string))
        print("average codeword length:",huffman_length(input_string))
        print("joint average length:",joint_huffman_length(input_string))



In [15]:
def first_order_entropy(string):
    # TODO:
    # Function to calculate first-order entropy
    n = len(string)
    frequency = {}
    for char in string:
        if char in frequency:
            frequency[char] += 1
        else:
            frequency[char] = 1
    entropy = -sum((char_count / n) * math.log2(char_count / n) for char_count in frequency.values())

    return entropy

# NOTE: Second order entropy function returns the value as per symbol, not per 2 symbols
def second_order_entropy(string):
    # TODO:
    # Function to calculate second-order entropy
    joint_chars = np.array([string[i:i+2] for i in range(0, len(string) - 1, 2)])
    unique_pairs, counts = np.unique(joint_chars, return_counts=True)

    frequency = {joint_char: count for joint_char, count in zip(unique_pairs, counts)}
    n = len(joint_chars)
    entropy = (-sum((pair_count / n) * math.log2(pair_count / n) for pair_count in frequency.values())) / 2
    return entropy


def huffman_length(string):
    # TODO:
    # Function to calculate average codeword length
    avg_length = 0
    n = len(string)
    frequency = {}
    for char in string:
        if char in frequency:
            frequency[char] += 1
        else:
            frequency[char] = 1

    heap = [[freq, [char, ""]] for char, freq in frequency.items()]
    heap.sort()

    while len(heap) > 1:
        lo = heap.pop(0)  # Smallest frequency node
        hi = heap.pop(0)  # Second smallest frequency node

        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]

        merged = [lo[0] + hi[0]] + lo[1:] + hi[1:]
        heap.append(merged)
        heap.sort()

    huffman_dict = dict(heap[0][1:])
    avg_length = sum(len(huffman_dict[char]) * frequency[char] for char in frequency) / n
    return avg_length


def joint_huffman_length(string):
    # TODO:
    # Function to calculate average codeword length for 2 symbol joint Huffman coding
    n = len(string)

    joint_chars = [string[i:i+2] for i in range(0, n - 1, 2)]

    frequency = {}
    for pair in joint_chars:
        if pair in frequency:
            frequency[pair] += 1
        else:
            frequency[pair] = 1

    heap = [[freq, [pair, ""]] for pair, freq in frequency.items()]
    heap.sort(key=lambda x: x[0])

    while len(heap) > 1:
        lo = heap.pop(0)  # Smallest frequency node
        hi = heap.pop(0)  # Second smallest frequency node

        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]

        merged = [lo[0] + hi[0]] + lo[1:] + hi[1:]
        heap.append(merged)
        heap.sort(key=lambda x: x[0])

    huffman_dict = dict(heap[0][1:])
    total = len(joint_chars)
    joint_avg_length = (sum(len(huffman_dict[pair]) * frequency[pair] for pair in frequency) / total) / 2

    return joint_avg_length



In [16]:
Q3_test()

input string:  AABB
first order entropy: 1.0
second order entropy: 0.5
average codeword length: 1.0
joint average length: 0.5


In [ ]:
# (3) Discuss any issues/interesting observations in your implementation.
# NOTE: My second order entropy function returns the value as per symbol, not per 2 symbols
# 2-joint huffman coding typically performs much better in terms of average codeword length per symbol.
# In most cases this is true, I will be doing experiments below to find more details about the performance differences
# between huffman coding and joint huffman coding.
# The single order entropy and average codeword length of single symbol huffman coding are very close to one another.
# The same applies to second order entropy and 2 joint huffman coding
# Much like how it is described in our lecture slides, the entropy is always below or equal its corresponding huffman coding's
# average codeword length. H(S) <= L < H(S) + 1 is always maintained


In [17]:
# (4) Experiment different randomly/non randomly generated inputs, and discuss your observations.
# You can implement your experiment here
import random

def generate_random_string(max_length=32):
    length = random.randint(2, max_length // 2) * 2  # Ensure even length
    return ''.join(random.choices('ABCDE', k=length))

ran_string = generate_random_string()
defined_string = "AAABBAABABAABA"

print("Random Tests:")
print("String: ", ran_string)
print("first order entropy:",first_order_entropy(ran_string))
print("average codeword length:",huffman_length(ran_string))
print("second order entropy:",second_order_entropy(ran_string))
print("joint average length:",joint_huffman_length(ran_string))
print("\n")

print("Pre-defined Tests:")
print("String: ", defined_string)
print("first order entropy:",first_order_entropy(defined_string))
print("average codeword length:",huffman_length(defined_string))
print("second order entropy:",second_order_entropy(defined_string))
print("joint average length:",joint_huffman_length(defined_string))

# From experiments, I can see that strings with a higher entropy (more random order of letters) and a more even
# distribution of symbols (e.g. ABCD, where each symbol is equally likely to appear) do worse in regular huffman coding.
# These kinds of inputs do much better in 2-symbol joint huffman coding and the disparity between average word lengths is
# much greater in these scenarios. Considering it logically, it makes sense that single symbol huffman coding
# performs poorly in these cases. If each symbol is of a near equal frequency, most symbols will be contained in the
# same lowest depth of the huffman coding tree. This will cause codeword lengths to be long and less efficient.

# String like the ones mentioned in the above explanation work better with 2-joint huffman coding likely because it reduces
# the total number of symbols by pairing up the letters. 2-joint coding also works better if symbols within strings are more
# dependent on their previous symbol. An example of this would be: BEBECAEBCAECEC, where pairs of characters are very
# frequently repeating. This causes 2-joint huffman coding to perform very well.

# Regular Huffman coding works best with inputs that have a small alphabet or when there is little contextual relationship
# between symbols. For the string to be optimal for huffman coding it would also need an uneven distribution of probability
# across symbols. Take this string for example: AAABBAABABAABA. The alphabet for it is small and therefore causes the
# the single symbol huffman encoding to perform much closer to the joint symbol huffman coding.

# Overall, 2-joint huffman coding performs better on average and has a lower average codeword length. However, the alphabet
# given in this assignment is quite limited, so in scenarios where the alphabet can be much more expansive, this conclusion
# might change.

Random Tests:
String:  CDCEBAABDA
first order entropy: 2.2464393446710154
average codeword length: 2.3
second order entropy: 1.160964047443681
joint average length: 1.2


Pre-defined Tests:
String:  AAABBAABABAABA
first order entropy: 0.9402859586706311
average codeword length: 1.0
second order entropy: 0.7783283537314114
joint average length: 0.7857142857142857
